In [ ]:
pip install geopandas shapely gpxpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
import os
import gpxpy
import geopandas as gpd
from shapely.geometry import LineString
import zipfile

def gpx_to_geodataframe(gpx_file):
    with open(gpx_file, 'r') as file:
        gpx = gpxpy.parse(file)

    lines = []  # List to hold LineString geometries
    for track in gpx.tracks:
        all_coords = []  # List to hold all coordinates for this track
        for segment in track.segments:
            # Create LineString from segment points
            coords = [(point.longitude, point.latitude) for point in segment.points]
            all_coords.extend(coords)  # Add segment coordinates to the track list

        if all_coords:  # Ensure there are coordinates to create a LineString
            line = LineString(all_coords)
            lines.append({
                'geometry': line,
                'elevation': [point.elevation for point in segment.points],  # Optional: Add elevation
                'time': [point.time for point in segment.points]  # Optional: Add time
            })

    return gpd.GeoDataFrame(lines, crs="EPSG:4326")  # Set the CRS for WGS84

def merge_gpx_files(gpx_files):
    all_lines = []
    for gpx_file in gpx_files:
        gdf = gpx_to_geodataframe(gpx_file)
        all_lines.append(gdf)

    return gpd.GeoDataFrame(pd.concat(all_lines, ignore_index=True))

def save_shapefile_as_zip(gdf, shapefile_name, output_folder):
    # Construct the shapefile path
    shapefile_path = os.path.join(output_folder, f"{shapefile_name}.shp")

    # Save as shapefile
    gdf.to_file(shapefile_path, driver="ESRI Shapefile")

    # Zip the shapefile components
    shapefile_zip = f"{shapefile_path}.zip"
    with zipfile.ZipFile(shapefile_zip, 'w') as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                if file.startswith(shapefile_name) and file.endswith(('.shp', '.shx', '.dbf', '.prj')):
                    zipf.write(os.path.join(root, file), arcname=file)

In [ ]:
# List of GPX files to merge
gpx_files = ['/content/Anastasia State Park Marsh Trail.gpx',
             '/content/Ancient Dunes Nature Trail.gpx',
             '/content/Cape Francis Beach Walk.gpx',
             '/content/Fort Matanzas Nature Trail.gpx',
             '/content/Fort Matanzas Nature Trail.gpx',
             '/content/Moses Creek White Trail.gpx',
             '/content/Moses Creek to Murats Point.gpx',
             '/content/Spanish Quarry.gpx',
             '/content/St. Augustine Walking Tour.gpx',
             '/content/Treaty Park Loop.gpx',
             '/content/Twelve Mile Swamp Conservation Area Trail.gpx',
             '/content/White Trail to Group Camping Site.gpx']  # Add your GPX file paths here
output_folder = "/content/output_folder"  # Specify your output directory
shapefile_name = "staug_10alltrails_data"  # Specify your shapefile name

# Merge GPX files and save as a shapefile zip
merged_gdf = merge_gpx_files(gpx_files)
save_shapefile_as_zip(merged_gdf, shapefile_name, output_folder)


/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Value '[1.0, 1.96, 1.67, 1.78, 1.67, 1.45, 1.02, 1.0, 1.1, 1.1, 1.0, 1.43, 1.21, 1.55, 1.12, 1.34, 1.27, 1.12, 1.07, 1.0, 1.13, 1.07, 1.0, 1.0, 1.0, 1.0, 1.0, 1.98, 2.0, 2.0, 2.0, 2.35, 3.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08, 0.0, 1.0, 1.25, 1.5, 1.64, 1.55, 1.85, 2.17, 1.85, 1.55, 1.64, 1.5, 1.25, 1.0, 0.0, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0, 3.03, 2.35, 2.0, 2.0, 2.0, 1.98, 1.0, 1.0, 1.0, 1.0, 1.0, 1.07, 1.13, 1.0, 1.07, 1.12, 1.27, 1.34, 1.12, 1.55, 1.21, 1.43, 1.0, 1.1, 1.1, 1.0, 1.02, 1.45, 1.67, 1.78, 1.67, 1.96, 1.0]' of field elevation has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


In [ ]:
from google.colab import files
files.download('/content/output_folder/staug_10alltrails_data.shp.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>